In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
fapeal = pd.DataFrame()

links_proyectos = []
fechas_proyectos = []
titulos_proyectos = []
pdfs_proyectos = []

estados = ["https://www.fapeal.br/category/editais/editais-vigentes/", "https://www.fapeal.br/category/editais/editais-encerrados/", "https://www.fapeal.br/category/editais/chamadas-abertas/"]


for estado in estados:



    #paginaciṕm
    parser_estado = parser(estado)
    paginacion = parser_estado.xpath('//div[@class="nav-links"]//text()')
    pag = []
    for i in paginacion:
        if i.isdigit():
            pag.append(i)
    
    pag_max = sorted(pag, reverse=True)[0]

    n = 1
    while n <= int(pag_max):

        end = "page/"+ str(n) +"/"

        if n == 1:
            parser_proyectos = parser(estado)
        else:
            parser_proyectos = parser(estado+end)

        titulos = []
        fechas = []
        links = []

        #link
        links = sorted(list(set(parser_proyectos.xpath('//div[@class="list-left-wrap"]//a/@href'))),reverse=True)

        #Titulos
        tits = parser_proyectos.xpath('//div[@class="list-left-wrap"]//h2[@class="entry-title"]/a/text()')
        for tit in tits:
            titulos.append(tit.strip())

        #Fechas
        fechas = parser_proyectos.xpath('//div[@class="list-left-wrap"]//div[@class="entry-meta"]/span/text()')

        for fecha in fechas:
            if fecha.isdigit():
                fechas.remove(fecha)
        
        n+=1

        links_proyectos+=links
        fechas_proyectos+=fechas
        titulos_proyectos+=titulos

# pdfs
for link in links_proyectos:

    parser_proyecto = parser(link)

    pdfs_proyecto = ''

    pdfs = parser_proyecto.xpath('//a/@href')

    for pdf in pdfs:
        if pdf.endswith('.pdf') or pdf.startswith('http://resultado.cnpq.br/'):
            pdfs_proyecto = pdfs_proyecto + pdf + ', '

    pdfs_proyectos.append(pdfs_proyecto.strip(', '))

fapeal['Titulo'] = titulos_proyectos
fapeal['Fecha'] = fechas_proyectos
fapeal['link'] = links_proyectos
fapeal['pdfs'] = pdfs_proyectos

#fapeal.to_excel('brasil_Fapeal.xlsx')

/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fapeal.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
fapeal.to_excel('brasil_Fapeal.xlsx')